<a href="https://colab.research.google.com/github/ryan-hayden16/Projects/blob/main/7_22_iterativeMLE_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preliminary steps

In [2]:
# imports and installs
!pip install scanpy # tools for scRNA-seq analysis
!pip install matplotlib==3.1.3 # current version produces error w/ scanpy
!pip install sklearn # tools for general data analysis

import pandas as pd
import numpy as np
import scanpy as sc
import sklearn
import matplotlib.pyplot as plt
import math
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

!pip install matplotlib==3.1.3 # reinstall to force old package version

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 3.6 MB/s 
     |████████████████████████████████| 96 kB 2.8 MB/s 
     |████████████████████████████████| 88 kB 8.7 MB/s 
     |████████████████████████████████| 11.2 MB 5.9 MB/s 
     |████████████████████████████████| 944 kB 49.3 MB/s 
     |████████████████████████████████| 1.1 MB 55.8 MB/s 
     |████████████████████████████████| 63 kB 1.1 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=d9daa7f7790faad4af42806f25587d8634b135ca17569f8fa2f04a8ae509a9d1
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=b757eefc4b33a381818514e27782b97365b6f8b2329f7cbef2ebc446d30199e7
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 5.9 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.2
    Uninstalling matplotlib-3.5.2:
      Successfully uninstalled matplotlib-3.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scanpy 1.9.1 requires matplotlib>=3.4, but you have matplotlib 3.1.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Load data (and metadata, if available)

In [ ]:
# load data matrix (single patient/sample)
x = pd.read_csv("/content/Kidney-counts.csv", index_col=0)
x = np.transpose(x) # transpose into cell by gene format

In [ ]:
# load metadata (if available, can be used to check cluster accuracy)
metadata = pd.read_csv("/content/annotations_FACS.csv", index_col=0)
metadata = metadata.loc[metadata['tissue'].isin(['Kidney'])]

In [ ]:
# remove raw data with missing labels
cellclass=metadata.cell_ontology_class
cellclass=cellclass.to_frame()
mergedf=x.merge(cellclass, left_index=True, right_index=True)
metadf=mergedf.cell_ontology_class
metadf=metadf.to_frame()
bladf=mergedf.drop(columns=['cell_ontology_class'])

# now raw data and metadata have matching sizes
x=bladf
x_labels=metadf

# create annotated data matrix (ie: anndata) to use with scanpy
adata_raw = sc.AnnData(X = x, obs = x_labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  


Quality control of raw data (optional)

In [ ]:
# quality control of raw data? (need old matplotlib version to avoid errors)

# quality control
adata_qc=adata_raw # keep copy of the raw data
is_spike_in = {}
for gene_name in adata_qc.var_names:
    if 'ERCC' in gene_name:
        is_spike_in[gene_name] = True # record that we found a spike-in
    else:
        is_spike_in[gene_name] = False # record that this was not a spike-in
adata_qc.var['ERCC'] = pd.Series(is_spike_in) # label the spike ins
qc = sc.pp.calculate_qc_metrics(adata_qc, qc_vars = ['ERCC']) # scanpy function
cell_qc_dataframe = qc[0] # cell quality control
gene_qc_dataframe = qc[1] # gene quality control

# cell filtering and gene filtering
low_ERCC_mask = (cell_qc_dataframe['pct_counts_ERCC'] < 10)
adata_qc = adata_qc[low_ERCC_mask]
sc.pp.filter_cells(adata_qc, min_genes = 750) # filter cells 
sc.pp.filter_genes(adata_qc, min_cells = 2) # filter genes
sc.pp.filter_genes(adata_qc, min_counts = 10)

#run PCA with no labels
sc.pp.pca(adata_qc)
sc.pl.pca_overview(adata_qc) # plot

# run PCA as exploratory measure to check the data out
sc.pp.pca(adata_qc)
sc.pl.pca_overview(adata_qc, color='cell_ontology_class') # plot

# normalize the data 
adata_norm=adata_qc # keep copy of qc data
sc.pp.normalize_per_cell(adata_norm, counts_per_cell_after=1e6)
sc.pp.normalize_total(adata_norm, target_sum=1e6, exclude_highly_expressed=True)

# (OPTIONAL) Remove highly expressed genes distorting the data
not_Rn45s = adata_norm.var.index != 'Rn45s'
adata_no_Rn45s = adata_norm[:, not_Rn45s] # keep copy of normed data
# need to check which genes to remove

# scale the data
adata_scale=adata_no_Rn45s
# adata_scale=adata_norm
sc.pp.log1p(adata_scale)
sc.pp.scale(adata_scale)

#re-run PCA with no labels
sc.pp.pca(adata_scale)
sc.pl.pca_overview(adata_scale) # plot

# re-run PCA, should seperate data better this time
sc.pp.pca(adata_scale)
sc.pl.pca_overview(adata_scale, color='cell_ontology_class') # plot

adata=adata_scale # adata is now quality controlled, normalized, and scaled

Extract count matrix from raw data

In [ ]:
# convert data matrix in adata to dataframe
x = pd.DataFrame(adata.X)
x=x.set_index(adata.obs.index)

#NOTE, numpy ndarray is preferable to dataframe because it can be more than 2 dimensions, but tensorflow tensor might be computationally advantageous


Define known variables

In [ ]:
# define known values
C,G = x.shape # retrieve number of cells and genes from raw data matrix
K = 5 # predicted number of cell-types
L = 3 # predicted number of gene-communities (ie: high, medium, low expression communities)


# define distribution f (start with Poisson) 
poisson pdf

python notes

In [ ]:
#create tensor in python
import random
C=50
G=100
K=5 
L=3 
Q=np.zeros((G,K,L))
for i in range(G):
  for j in range(K):
    for l in range(L):
      Q[i,j,l]=random.uniform(0, 1)

print(Q)
# Q is GxKxL tensor

In [4]:
# generate fake (normalized) count data
X=np.zeros((C,G))
for c in range(C):
  for g in range(G):
    X[c,g]=random.uniform(0,1)

In [45]:
X # maybe multiply each entry * 100, to avoid numerical errors????

array([[0.21326413, 0.61979383, 0.23554814, ..., 0.37794462, 0.35046691,
        0.3547858 ],
       [0.92062858, 0.47571902, 0.3754333 , ..., 0.30913323, 0.63323187,
        0.99553775],
       [0.29627568, 0.09308347, 0.04824277, ..., 0.4236403 , 0.81682611,
        0.67860941],
       ...,
       [0.45548368, 0.44814593, 0.6696283 , ..., 0.40853438, 0.05663819,
        0.22891408],
       [0.70546348, 0.47833187, 0.54035059, ..., 0.51452846, 0.68232088,
        0.69154152],
       [0.1937172 , 0.36486966, 0.72162352, ..., 0.4815606 , 0.58059982,
        0.88703628]])

In [15]:
np.set_printoptions(threshold=np.inf)
# force python to print entire array

Initialize S and T variables

In [5]:
# initialize S
import scipy as sp
from numpy import linalg as LA
from scipy.linalg import sqrtm
from numpy.linalg import inv
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans


A=np.dot(X,np.transpose(X)) #step 1 (affinity matrix, simple version)

D=np.zeros((C,C))
for i in range(C):
  D[i,i] = sum(A[i,:]) #step 2.a (graph laplacian)

E=sqrtm(D) 

# note that np.dot(E,E)==D returns a few falses, possibly because of numerical error? should be identical

F=inv(E) 
H=np.dot(F,np.dot(A,F)) #step 2.b (graph laplacian)

# note that H==np.transpose(H) returns some falses, ie: not symmetric yet we get all real eigenvalues here

w, v = LA.eig(H) #step 3 (find K largest (orthogonal) eigenvectors of H and form a CxK matrix with them) and normalize the matrix
ordered_eigval=np.argsort(w) # returns indexes of ordered (small to large) of w (eigenvalue list)
k_large_eigval = ordered_eigval[-K:] # returns indexes of K largest eigvals
k_large_eigvec=np.transpose(v[k_large_eigval]) # returns corresponding K largest eigvecs, as a CxK ndarray
Y = normalize(k_large_eigvec, axis=1, norm='l2') #normalize rows of the CxK matrix


#step 5 (treat each of the C rows of the matrix as a K-dim vector and cluster into K-clusters, via K-means)
kmeans = KMeans(n_clusters=K, random_state=0).fit(Y)
cluster_labels = kmeans.labels_

#step 6 (assign/label each of the C cells into the corresponding cluster (ie: labels are 1,2,...,K) from step 5)
S=np.zeros((C,K))
for i in range(C):
  for j in range(K):
    if cluster_labels[i]==j:
      S[i,j]=1
#(each cell now has a label from 1,...,K, so we can then form the CxK classification matrix (ie: matrix S) that we want)


# this process gives us S_0

# THIS CODE IS FINISHED AND HAS BEEN CHECKED TO BE WORKING PROPERLY

In [ ]:
cluster_labels

In [8]:
S.shape

(50, 5)

In [6]:
# initialize T

T=np.zeros((G,K,L))

for i in range(K):
  XT=X
  XT=np.delete(XT,np.where(cluster_labels!=i),0) #now XT should only contain rows with label i
  A=np.dot(np.transpose(XT),XT) #step 1 (affinity matrix, simple version)
  D=np.zeros((G,G))
  for l in range(G):
    D[l,l] = sum(A[:,l]) #step 2.a (graph laplacian)
  E=sqrtm(D) 
  F=inv(E) 
  H=np.dot(F,np.dot(A,F)) 

# H should be symmetric, so shouldnt even have complex eigenvectors

  w, v = LA.eig(H) #step 3 (find K largest (orthogonal) eigenvectors of H and form a GxK matrix with them) and normalize the matrix
  ordered_eigval=np.argsort(w) # returns indexes of ordered (small to large) of w (eigenvalue list)
  k_large_eigval = ordered_eigval[-K:] # returns indexes of K largest eigvals
  k_large_eigvec=np.transpose(v[k_large_eigval]) # returns corresponding K largest eigvecs, as a GxK ndarray
  complex_k_large_eigvec=np.array(k_large_eigvec, dtype = 'complex_') # convert to complex valued matrix? does not fix error
  e=LA.norm(complex_k_large_eigvec, axis=1) #
  for n in range(C):
    complex_k_large_eigvec[n,:]=complex_k_large_eigvec[n,:]/e[n]
  Y=complex_k_large_eigvec
  B=Y.real # possible solution: just consider real part (do this in S initialization as well)
  # whatever solution is, apply to S initialization as well in case its eigenvalues are complex
  #try dividing each vector in k_large_eigvec by the corresponding value in LA.norm(k_large_eigvec), ie manually scale to unit vector
  # Y = normalize(complex_k_large_eigvec, axis=1, norm='l2') #normalize rows of the GxK matrix (CAUSES ERROR DUE TO COMPLEX NUMBERS)
  #step 5 (treat each of the C rows of the matrix as a K-dim vector and cluster into K-clusters, via K-means)
  #above solution seemed to work, but no error is in k-means

# ASK YUNPENG HOW TO DEAL WITH K-MEANS IF VECTORS ARE COMPLEX VALUED ??? MAYBE JUST DROP IMAGINARY PART? OR TAKE MAGNITUDE? ITS ONLY AN INITIAL GUESS FOR S

  kmeans = KMeans(n_clusters=L, random_state=0).fit(B)
  Tcluster_labels = kmeans.labels_
  #step 6 (assign/label each of the C cells into the corresponding cluster (ie: labels are 1,2,...,K) from step 5)
  for g in range(G):
    for l in range(L):
      if Tcluster_labels[g]==l:
        T[g,i,l]=1

# ALSO MAKE SURE TO SWAP TRANSPOSE ORDER SO ITS NOW GxG AFFINITY MATRIX, AND SWAP WHEREVER ELSE NECCESARY, IE NOW K-MEANS ON COLUMNS AND NORMALIZE COLUMNS, ETC...


In [ ]:
T

In [7]:
T.shape

(100, 5, 3)

In [ ]:
#now code seems to work, but clean it up, make sure no logic errors, and change S initialization code so its also able to deal with complex #'s

notes/errors

In [ ]:
e=LA.norm(k_large_eigvec, axis=1)
for i in range(C):
  k_large_eigvec[i,:]=k_large_eigvec[i,:]/e[i]
#this can replace sklearn l2 normalizer, check below for accuracy

In [52]:
XT=X
XT=np.delete(XT,np.where(cluster_labels!=0),0) #now XT should only contain rows with label i
A=np.dot(np.transpose(XT), XT) #step 1 (affinity matrix, simple version)
D=np.zeros((G,G))
for g in range(G):
  D[g,g] = sum(A[:,g]) #step 2.a (graph laplacian)
  # maybe change g to column instead of row
E=sqrtm(D) 
F=inv(E) 
H=np.dot(F,np.dot(A,F)) 
w, v = LA.eig(H)

Initialize theta paramaters

In [46]:
mu=np.zeros((K,L))
for k in range(K):
  for l in range(L):
    numer=0
    denom=0
    for c in range(C):
      for g in range(G):
        numer=numer+(S[c,k]*T[g,k,l]*X[c,g])
        denom=denom+(S[c,k]*T[g,k,l])
    mu[k,l]=numer/denom

In [ ]:
numer=0
denom=0
  for c in range(C):
    for g in range(G):
      numer=numer+(S[c,k]*T[g,k,l]*X[c,g])
      denom=denom+(S[c,k]*T[g,k,l])

In [47]:
mu

array([[0.49497398, 0.50071955, 0.50456214],
       [0.48855569, 0.49526875, 0.5103612 ],
       [0.5068372 , 0.49999404, 0.46634825],
       [0.52030205, 0.49395987, 0.50169942],
       [0.51902416, 0.48830684, 0.50092061]])

In [ ]:
# using S_0 and T_0 and X, we can directly calculate rho_0, pi_0, and mu_0

#rho: use yunpeng line (12) and S_0
rho=np.zeros(K)
for k in range(K):
  rho[k]=(1/C)*sum(S[:,k])

#pi: use yunpeng line (13) and T_0
pi=np.zeros((K,L))
for k in range(K):
  for l in range(L):
    pi[k,l]=(1/G)*sum(T[:,k,l])


#mu: use yunpeng line (14) and S_0, T_0, X 
mu=np.zeros((K,L))
for k in range(K):
  for l in range(L):
    numer=0
    denom=0
    for c in range(C):
      for g in range(G):
        numer=numer+(S[c,k]*T[g,k,l]*X[c,g])
        denom=denom+(S[c,k]*T[g,k,l])
    mu[k,l]=numer/denom

# double check logic of mu loop

Iterative maximization scheme

In [ ]:
# EM steps

S_old = 0
S_new = S

T_old = 0
T_new = T

epsilon_1 = .01
epsilon_2 + .01

while ((S_new - S_old > epsilon_1) && (T_new - T_old > epsilon_2)) #something like this, maybe also check theta convergence???
# note that S and T are matrices so cant just subtract, need to take norm or something, 
# ask yunpeng best way to measure S and T convergence

# update S, rho, mu

S_old=S_new # save copy of previous S update
for c in range(C):
  for k in range(K):
    S_new[c,k]= # update S, using updated rho, mu, and T values
    # S_NEW IS LINE (9)

for k in range(K):
  rho[k]=(1/C)*sum(S_new[:,k]) # update rho, using updated S values

for k in range(K):
  for l in range(L):
    numer=0
    denom=0
    for c in range(C):
      for g in range(G):
        numer=numer+(S_new[c,k]*T_new[g,k,l]*X[c,g])
        denom=denom+(S_new[c,k]*T_new[g,k,l])
    mu[k,l]=numer/denom # update mu, using updated S and T values


# update T, pi, mu

T_old=T_new # save copy of previous S update
for g in range(G):
  for k in range(K):
    for l in range(L):
      T_new[g,k,l]= # update T, using updated pi, mu, and S values
      # T_NEW IS LINE (10)

for k in range(K):
  for l in range(L):
    pi[k,l]=(1/G)*sum(T_new[:,k,l]) # update pi, using updated T values

for k in range(K):
  for l in range(L):
    numer=0
    denom=0
    for c in range(C):
      for g in range(G):
        numer=numer+(S_new[c,k]*T_new[g,k,l]*X[c,g])
        denom=denom+(S_new[c,k]*T_new[g,k,l])
    mu[k,l]=numer/denom # update mu, using updated S and T values

In [ ]:
# start with S_0, T_0, rho_0, pi_0, mu_0

# loop through steps 1 and 2


#NOTE: be careful with regard to dropouts/blowups, use appropriate numerical techniques


# step 1: update S, rho, mu
fix previous values of S, T, rho, pi, mu 
calculate updated S # E step (define function) 
USE YUNPENG LINE (9)
calculate updated rho, mu # M step (define function)
USE YUNPENG LINES (12 & 14)

# step 2: update T, pi, mu
fix previous values of S, T, rho, pi, mu 
calculate updated T # E step (define function)
USE YUNPENG LINE (10)
calculate updated pi, mu # M step (define function)
USE YUNPENG LINES (13 & 14)

# stop loop if convergence criteria met, ie: S and T both no longer changing values to significant degree
IE: T_(M+1)-T_(M) < epsilon, and similar for S_(M)

In [ ]:
# convert S and T into labels Z and W
# check Z (cell-type) labels against true labels from metadata
# check high expression gene-community against known biomarkers for associated cell-type

In [ ]:
# figure out way to visualize Z and W labels